# LASSO using Coordinate Descent Optimization Alogrithm

In this notebook, we will implement our very own LASSO solver via coordinate descent. We will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

In [223]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [224]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

# Import useful functions from previous notebook

# Convert to Numpy Array


Although SFrames offer a number of benefits to users (especially when using Big Data and built-in graphlab functions) in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

First we need to take the SFrame of our data and convert it into a 2D numpy array (also called a matrix). To do this we use graphlab's built in .to_dataframe() which converts the SFrame into a Pandas (another python library) dataframe. We can then use Panda's .as_matrix() to convert the dataframe into a numpy matrix.

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:

A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
A numpy array containing the values of the output

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features 
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return feature_matrix, output_array

# Predicting output given regression weights

In [5]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** : we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, we will write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function is going to return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. We will use these norms to normalize the test data in the same way as we normalized the training data. 

# Function to Normalize the dataset

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis = 0)
    nomalized_features = feature_matrix / norms
    return nomalized_features, norms

Testing the function, by running the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (we would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times. We will:  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output(simple_feature_matrix, weights)

We will compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```
We can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

# Computing Rho

In [109]:
def compute_rho(feature_matrix, output, weights, prediction, i):
    return sum(feature_matrix[:, i]*(output - prediction + weights[i] * feature_matrix[:, i]))

In [102]:
ro = {}
for i in range(3):
    ro['W_'+str(i)] = compute_rho(simple_feature_matrix, output, weights, prediction, i)

In [103]:
ro

{'W_0': 79400300.03492953, 'W_1': 87939470.7729908, 'W_2': 80966698.67596525}

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [104]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_output(feature_matrix, weights)
    ro_i = compute_rho(feature_matrix, output, weights, prediction, i)

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + (l1_penalty / 2)
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - (l1_penalty / 2)
    else:
        new_weight_i = 0.
    
    return new_weight_i

For testing the function, we will run the following cell:

In [110]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [150]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    converged = False
    weights = np.array(initial_weights)
        
    while not converged:
        weight_diff_list = []
        for i in range(len(weights)):
            
            old_weights = weights[i]
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            
            weight_difference = np.abs(new_weight - old_weights)
            weights[i] = new_weight

            weight_diff_list.append(weight_difference)
                
        if max(weight_diff_list) < tolerance:
            converged = True
    
    return weights

In [112]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First creating a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [113]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, running our implementation of LASSO coordinate descent:

In [152]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [169]:
print "Weight of Intercept: %0.2e"%weights[0]+"\nWeight of Sqft_living: %0.2e"%weights[1]+"\nWeigth of bedroom:"+str(weights[2])

Weight of Intercept: 2.16e+07
Weight of Sqft_living: 6.32e+07
Weigth of bedroom:0.0


# Computing RSS

In [216]:
def compute_rss(feature_matrix, weigths, output):
    predictions = np.dot(feature_matrix, weights)
    error = predictions - output
    rss = sum(error**2)
    
    return rss

In [163]:
print("Residual Sum of Squares is: %.2e" % compute_rss(normalized_simple_feature_matrix, weights, output))

Residual Sum of Squares is: 1.63e+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [170]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [171]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, creating a normalized feature matrix from the TRAINING data with these features.  (Make sure to store the norms for the normalization, since we'll use them later)

In [180]:
my_output = 'price'
feature_matrix, output = get_numpy_data(train_data, all_features, my_output)
normalized_features, norms = normalize_features(feature_matrix)

In [181]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0

First, learning the weights with `l1_penalty=1e7`, on the training data. Initializing weights to all zeros, and setting the `tolerance=1`.  Call resulting weights `weights1e7`, as we will need them later for comparing purpose.

# Computing weights for L1_penalty = 1e7

In [184]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_features, output, initial_weights, l1_penalty, tolerance)

In [193]:
weights1e7

array([24429600.60933346,        0.        ,        0.        ,
       48389174.35227959,        0.        ,        0.        ,
        3317511.16271979,  7329961.98489638,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [201]:
print 'constant', '\t\t', weights1e7[0]
for feature, weight in zip(all_features, weights1e7[1:]):
    print feature, '\t\t', weight

constant 		24429600.60933346
bedrooms 		0.0
bathrooms 		0.0
sqft_living 		48389174.35227959
sqft_lot 		0.0
floors 		0.0
waterfront 		3317511.1627197945
view 		7329961.984896379
condition 		0.0
grade 		0.0
sqft_above 		0.0
sqft_basement 		0.0
yr_built 		0.0
yr_renovated 		0.0


Next, we will learn the weights with `l1_penalty=1e8`, on the training data. Initializing weights to all zeros, and settinh the `tolerance=1`.  Call it the resulting weights `weights1e8`.

# Computing weights for L1_penalty = 1e8

In [202]:
l1_penalty = 1e8
initial_weights = np.zeros(len(all_features)+1)
weights1e8 = lasso_cyclical_coordinate_descent(normalized_features, output, initial_weights, l1_penalty, tolerance)

In [203]:
print 'constant', '\t\t', weights1e8[0]
for feature, weight in zip(all_features, weights1e8[1:]):
    print feature, '\t\t', weight

constant 		71114625.75280942
bedrooms 		0.0
bathrooms 		0.0
sqft_living 		0.0
sqft_lot 		0.0
floors 		0.0
waterfront 		0.0
view 		0.0
condition 		0.0
grade 		0.0
sqft_above 		0.0
sqft_basement 		0.0
yr_built 		0.0
yr_renovated 		0.0


Finally, learning the weights with `l1_penalty=1e4`, on the training data. Initializing weights to all zeros, and setting `tolerance=5e5`.  Call resulting weights `weights1e4`. (This case will take quite a bit longer to converge than the others above.)

# Computing weights for L1_penalty = 1e4 & Tolerance = 5e5

In [204]:
l1_penalty = 1e4
tolerance = 5e5
initial_weights = np.zeros(len(all_features)+1)
weights1e4 = lasso_cyclical_coordinate_descent(normalized_features, output, initial_weights, l1_penalty, tolerance)

In [205]:
print 'constant', '\t\t', weights1e4[0]
for feature, weight in zip(all_features, weights1e4[1:]):
    print feature, '\t\t', weight

constant 		77779073.91265057
bedrooms 		-22884012.25023268
bathrooms 		15348487.08089926
sqft_living 		92166869.69883154
sqft_lot 		-2139328.082427727
floors 		-8818455.544094577
waterfront 		6494209.733106562
view 		7065162.050532017
condition 		4119079.210067577
grade 		18436483.526187778
sqft_above 		-14566678.545144046
sqft_basement 		-5528348.751794508
yr_built 		-83591746.20730412
yr_renovated 		2784276.460128562


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

# Normalizing weigths of above learned weights

In [207]:
normalized_weights1e4 = weights1e4 / norms 
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms

In [208]:
print normalized_weights1e7[3]

161.31745624837794


To check the results, if we call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [209]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

# Computing RSS for weight = 1e4

In [219]:
prediction = np.dot(test_feature_matrix, normalized_weights1e4)
error = prediction - test_output
RSS_1e4 = sum(error**2)

# Computing RSS for weight = 1e7

In [220]:
prediction = np.dot(test_feature_matrix, normalized_weights1e7)
error = prediction - test_output
RSS_1e7 = sum(error**2)

# Computing RSS for weight = 1e8

In [221]:
prediction = np.dot(test_feature_matrix, normalized_weights1e8)
error = prediction - test_output
RSS_1e8 = sum(error**2)

In [222]:
RSS_1e4, RSS_1e7, RSS_1e8

(227781004760224.62, 275962079909185.12, 537166150034084.1)

As we can clearly see from above that RSS is minimum for model having **weight = 1e4**

# Good Luck